In [ ]:
pip install -q yahoo_fin

In [ ]:
pip install --upgrade yfinance

In [3]:
mkdir -p data results log

In [4]:
from google.colab import drive
drive.mount('/content/drive')
# ! find /content/drive/MyDrive/colab/ -type f -mtime +180 -delete -print


Mounted at /content/drive


In [5]:
import numpy as np
import pandas as pd

In [6]:
def show_prediction(name, suffix):
  df = pd.read_csv(f'results/{name}_{suffix}_predictions.csv')
  df['predicted_return'] = ((df['predicted_max']+df['predicted_min']+df['predicted_close'])/3 - df['current_price'])/df['current_price']
  return df.sort_values('predicted_return', ascending=False, key=abs)


In [7]:
def run_command_with_check(command):
  """Runs a command, prints stderr, and raises an exception if it fails."""
  process = subprocess.run(command, shell=True, capture_output=True, text=True, check=False)

  if process.returncode != 0:
    print("stderr:", process.stderr)  # Print stderr only on error
    raise subprocess.CalledProcessError(process.returncode, process.args, output=process.stdout, stderr=process.stderr)
  else:
    # Print or process stdout if needed
    #print("stdout:", process.stdout)
    pass

  return process.returncode


In [8]:
import os
import subprocess
def run_model_nn_cmd(prefix, version, do_train=False, window_size=None, future_days=None, hourly=False, copy_tickers=True):
    # copy drive/MyDrive/colab/results/nnreturn.py to local folder
    tickers = f"{prefix}tickers.csv"
    run_command_with_check(f"cp  /content/drive/MyDrive/colab/nnreturn.py .")
    if copy_tickers:
      run_command_with_check(f"cp  /content/drive/MyDrive/colab/tickers/{tickers} .")
    if not do_train:
      run_command_with_check(f"cp  /content/drive/MyDrive/colab/models/{prefix}_{version}_*.pth .")
      # Run nnreturn.py in results folder and throw exception if error
      today = pd.to_datetime('today').strftime('%m%d')
      run_command_with_check(f"rename 's/{version}/{today}/' *.pth")
      cmd_arg = ""
      if window_size:
        cmd_arg += f" --window-size {window_size}"
      if future_days:
        cmd_arg += f" --future-days {future_days}"
      if hourly:
        cmd_arg += " --hourly"
      run_command_with_check(f"python3 nnreturn.py --tickers-file {prefix}tickers.csv --mode analyze {cmd_arg}")
      run_command_with_check(f"cp {prefix}*predictions.csv results")
      # Get today's date

      return show_prediction(prefix, today)
    else:
      raise Exception("Not implemented")


In [ ]:
run_model_nn_cmd('etf', '0730').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
0,ARKF,46.08,2026-01-27,2026-02-24,45.52,51.11,49.24,0.055194
2,ARKW,145.46,2026-01-27,2026-02-24,143.18,158.55,154.67,0.045877
4,DXYZ,29.50,2026-01-27,2026-02-24,27.87,31.54,32.95,0.043616
32,XSW,178.36,2026-01-27,2026-02-24,170.48,188.94,197.13,0.040125
10,IXP,123.00,2026-01-27,2026-02-24,114.36,125.10,115.54,-0.037940
1,ARKK,79.79,2026-01-27,2026-02-24,78.84,85.42,84.02,0.037223
15,JXI,81.94,2026-01-27,2026-02-24,80.53,88.54,84.11,0.029941
8,IXC,46.34,2026-01-27,2026-02-24,44.82,49.46,47.59,0.020501
14,JETS,27.83,2026-01-27,2026-02-24,25.94,28.93,26.97,-0.019763
9,IXN,109.84,2026-01-27,2026-02-24,104.04,115.55,116.41,0.019665


In [ ]:
# 0716 is hourly 20
run_model_nn('idx', '0716', window_size=20, future_days=8, hourly=True).head(10)

In [ ]:
import runml.pipeline
from runml import pipeline,findata
#from importlib import reload
# reload(runml.pipeline)

In [ ]:
# Skip some days from consecutive window to lessen test /train overlap
# findata.G_NON_OVERLAP= 0

Run following if like to save results in drive

In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    # Simplify following using apply
    def calculate_alloc(row):
        gain_low = min(row['Gain_l'], 0)
        gain_high = max(row['Gain_h'], 0)
        gain = min(row['Gain'], gain_high)
        average_gain = (gain_low+gain_high + 2*gain)/4
        if (average_gain == 0):
          return 0
        avg_accuracy = (row['Accu']+row['Accu_l']+row['Accu_h'])/3
        alloc =  avg_accuracy/stop_loss - (1-avg_accuracy)*profit_factor/abs(average_gain)
        return round(max(alloc, 0), 2)
    df['Alloc'] = df.apply(calculate_alloc, axis=1)
    return df

pd.options.display.max_columns = None

In [ ]:
import os
# findata.EPOCHS=200
def run_model(prefix, version, tickers, do_train):
  if (not do_train):
    os.system(f"cp  /content/drive/MyDrive/colab/results/{prefix}-{version}-* ./results/")
  mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(),
                         pipeline.AddMA(200), pipeline.Adj()]))
  df= pipeline.runModelCombinedVola(tickers, f"{prefix}-{version}", mod, do_train)
  # Save datafrme to prefix-version-prediction.csv
  df.to_csv(f"results/{prefix}-{version}-prediction.csv")
  return df



In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax' # Change to minmax
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BASE', 'BILL', 'BMBL', 'BYRN', 'CELH', 'CFLT', 'COHR', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ELF', 'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HCP', 'HIMS', 'HOOD', 'HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JAKK', 'JKS', 'KD', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR', 'MRVL', 'MXL','MTCH', 'MU',
           'NET', 'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PAR', 'PANW',  'PAYX', 'PD', 'PI', 'PINS',
           'RBLX', 'RKLB', 'RMBS', 'RUM', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'SOUN', 'TEM', 'TEAM', 'TDOC', 'TMDX', 'TNDM', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
df = run_model('ipos', '2a', tickers, True)

In [ ]:
addAllocHL(df, 0.10, 1)
df[(df.Gain>0)].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'CIBR', 'DAPP', 'DIA', 'DTEC', 'DXYZ',
            'EEM', 'FDN', 'FPX',
            'ICLN', 'IGV', 'IJK', 'IJR', 'IPO',
            'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'JXI', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'URA', 'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
df1 = run_model('etf', '1b', tickers1, True)



In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

In [ ]:
df1.to_csv('results/etf_12b_predictions.csv')

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax'
tickers3 = [ 'AIRI', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS',  'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CHGG', 'CLOV', 'CRCT', 'CXM',
            'DLO', 'DM', 'EVGO', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MITK', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PLUG', 'PTON', 'PUBM',
            'REAL', 'REI', 'RIG', 'RVYL',
            'SABR', 'SFIX', 'SLDP', 'SLE', 'SMWB', 'SST', 'STEM',
            'TDW', 'TDOC', 'TRUP', 'VTEX', 'VSAT', 'YEXT']
df3 = run_model('vols', '12b', tickers3, False)

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3[(df3.Gain>0)].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DELL', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'KO', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']
df4 = run_model('mcap', '1b', tickers4, True)

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
tickers5 = ['ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CX', 'DBOEY', 'DNNGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FDRVF', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']

df5 = run_model('gstock', '2a', tickers5, False)

In [ ]:
addAllocHL(df5, 0.14)
df5[df5.Gain > 0].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.G_SCALER = 'standard'
tickers6 = ['ARGT', 'EWD', 'FLN',
           'ECH', 'EPHE', 'EWA', 'EWG', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWT', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'KWT', 'THD', 'TUR', 'VNM', 'XCEM']
df6 = run_model('etf', '12a', tickers6, False)

In [ ]:
addAllocHL(df6, 0.07, 1)
df6.sort_values('Alloc', ascending=False)

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'standard'
tickers7 = ['AAL', 'ADI', 'ALB', 'AMR','ANF', 'APO', 'AQN', 'ARCH', 'ARE', 'ASC',
            'BAH', 'BAX', 'BKR', 'BLK', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IFF', 'IP', 'IPI', 'IVZ',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW', 'ZTS']
df7 = run_model('val', '1a', tickers7, False)

In [ ]:
addAllocHL(df7, 0.10, 1)
df7[df7.Gain > 0].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
tickers8 = [ 'DBC', 'FXB', 'FXE', 'FXF', 'FXY',
            'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'USO', 'UUP', 'VTIP', 'WEAT']

df8 =  run_model('com', '12a', tickers8, True)

In [ ]:
addAllocHL(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

In [ ]:
dfp = run_model_nn_cmd('pennystock', '1226')

In [ ]:
dfp[dfp['predicted_return'] > 0].head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
106,DVLT,0.70,2026-01-28,2026-02-25,0.56,1.26,0.71,0.204762
50,RVYL,7.20,2026-01-28,2026-02-25,6.60,10.13,8.43,0.164815
31,LFMD,3.46,2026-01-28,2026-02-25,3.07,4.30,4.60,0.153179
28,INVZ,1.06,2026-01-28,2026-02-25,0.98,1.27,1.37,0.138365
86,SMR,19.50,2026-01-28,2026-02-25,17.97,24.84,23.05,0.125812
42,PAYO,6.20,2026-01-28,2026-02-25,5.90,7.77,7.20,0.122043
54,SLE,6.55,2026-01-28,2026-02-25,5.99,8.71,7.27,0.118066
94,BYND,0.87,2026-01-28,2026-02-25,0.73,1.09,1.08,0.111111
109,AUID,1.48,2026-01-28,2026-02-25,1.20,2.08,1.64,0.108108
56,SST,4.17,2026-01-28,2026-02-25,3.79,5.22,4.72,0.097522


In [ ]:
run_model_nn_cmd('bluechipstock', '0221').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
12,COST,990.79,2026-02-04,2026-03-04,954.28,1102.57,1201.36,0.096166
25,LLY,1106.02,2026-02-04,2026-03-04,1062.97,1267.43,1206.89,0.066072
31,NOW,109.38,2026-02-04,2026-03-04,107.82,122.32,117.07,0.058115
21,INTC,47.90,2026-02-04,2026-03-04,43.05,47.97,44.53,-0.056715
27,MMC,188.49,2026-02-03,2026-03-03,188.15,199.59,203.11,0.044883
10,CDNS,268.09,2026-02-04,2026-03-04,265.91,281.08,285.72,0.035361
39,UNH,276.89,2026-02-04,2026-03-04,264.83,282.23,256.09,-0.033130
14,DE,562.89,2026-02-04,2026-03-04,554.39,609.22,580.16,0.032629
41,WMT,128.62,2026-02-04,2026-03-04,126.44,138.28,132.73,0.030037
32,ORCL,145.66,2026-02-04,2026-03-04,135.12,145.67,143.08,-0.030001


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'NEAR-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']
df9 = run_model('cry', '11b', tickers9, False)

In [ ]:
addAllocHL(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

In [ ]:
run_model_nn_cmd('country', '1112').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
23,VNM,18.03,2026-02-06,2026-03-06,16.98,18.18,17.72,-0.022370
18,INDA,53.28,2026-02-06,2026-03-06,50.26,53.86,52.47,-0.020333
20,KWT,37.25,2026-02-06,2026-03-06,35.10,38.37,36.35,-0.017271
10,EWM,29.57,2026-02-06,2026-03-06,27.96,31.30,30.87,0.016007
9,EWC,55.01,2026-02-06,2026-03-06,54.05,57.82,55.45,0.013876
11,EWP,56.46,2026-02-06,2026-03-06,55.25,59.98,56.38,0.013166
22,TUR,40.84,2026-02-06,2026-03-06,39.41,42.86,41.75,0.012243
19,KEMQ,25.92,2026-02-06,2026-03-06,24.58,26.43,25.80,-0.012217
5,EWA,28.23,2026-02-06,2026-03-06,27.37,29.54,28.81,0.012162
6,EWG,43.53,2026-02-06,2026-03-06,42.56,44.85,44.59,0.010797


In [ ]:
run_model_nn_cmd('global', '1107').head(15)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
3,BAK,3.48,2026-02-05,2026-03-05,3.37,3.97,3.93,0.079502
36,RACE,335.69,2026-02-05,2026-03-05,329.97,380.09,355.66,0.058238
60,TKAMY,13.14,2026-02-05,2026-03-05,12.09,15.03,14.57,0.057585
61,TRYIY,15.89,2026-02-04,2026-03-04,15.16,18.79,16.19,0.051815
47,OTGLY,17.20,2026-02-05,2026-03-05,14.68,17.92,16.34,-0.051550
31,IFNNY,48.44,2026-02-05,2026-03-05,42.22,48.60,47.44,-0.048582
6,BESIY,188.50,2026-02-05,2026-03-05,170.09,194.09,176.23,-0.044368
41,NHYDY,8.91,2026-02-05,2026-03-05,7.94,9.08,8.55,-0.043397
53,SFTBY,13.07,2026-02-05,2026-03-05,12.02,15.28,13.52,0.041061
16,EADSF,223.70,2026-02-05,2026-03-05,202.33,239.75,202.43,-0.039622


In [ ]:
run_model_nn_cmd('valuestock', '0301').head(15)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
41,FLO,11.96,2026-02-06,2026-03-06,11.79,13.51,13.50,0.081382
18,CCJ,113.63,2026-02-06,2026-03-06,85.67,118.47,110.88,-0.075890
64,OSK,171.71,2026-02-06,2026-03-06,145.67,174.38,158.15,-0.071691
1,ADI,320.45,2026-02-06,2026-03-06,286.59,320.40,288.06,-0.068966
71,POWL,584.22,2026-02-06,2026-03-06,516.15,587.87,532.03,-0.066533
96,WSM,220.67,2026-02-06,2026-03-06,194.74,218.50,213.69,-0.052990
86,TGT,115.55,2026-02-06,2026-03-06,103.90,116.84,108.32,-0.050743
60,NFG,85.77,2026-02-06,2026-03-06,85.40,94.97,88.77,0.045976
58,MOD,215.72,2026-02-06,2026-03-06,208.78,245.35,222.00,0.044765
99,ZTS,127.42,2026-02-06,2026-03-06,128.98,132.04,136.74,0.040548


In [ ]:
run_model_nn_cmd('commodity', '0312').head(5)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
8,SLV,75.87,2026-02-09,2026-03-09,74.01,83.13,75.20,0.020781
5,GLD,466.67,2026-02-09,2026-03-09,454.18,506.86,462.15,0.016557
12,USO,77.96,2026-02-09,2026-03-09,76.13,81.04,79.05,0.010005
4,FXY,58.99,2026-02-09,2026-03-09,57.68,61.41,59.22,0.007572
2,FXE,110.00,2026-02-09,2026-03-09,107.09,115.85,108.75,0.005121


In [ ]:
run_model_nn_cmd('crypto', '0214').head(5)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
17,ATOM-USD,1.96,2026-02-09,2026-03-09,1.89,2.99,2.31,0.222789
11,ICP-USD,2.45,2026-02-09,2026-03-09,2.21,2.68,4.08,0.220408
2,SOL-USD,87.85,2026-02-09,2026-03-09,86.80,132.35,98.71,0.206071
7,FIL-USD,0.94,2026-02-09,2026-03-09,0.84,1.45,1.01,0.170213
15,XLM-USD,0.16,2026-02-09,2026-03-09,0.16,0.19,0.17,0.083333


In [9]:
run_model_nn_cmd('techstock', '0312').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
58,PD,8.03,2026-02-10,2026-03-10,8.04,9.34,9.16,0.101702
67,SPOT,493.04,2026-02-10,2026-03-10,478.79,581.95,549.31,0.088519
30,GTLB,31.94,2026-02-10,2026-03-10,31.89,36.44,35.12,0.079628
76,U,29.07,2026-02-10,2026-03-10,22.77,30.86,27.26,-0.072469
8,BYRN,15.18,2026-02-10,2026-03-10,14.89,16.94,16.96,0.071366
52,NTNX,42.39,2026-02-10,2026-03-10,42.75,47.08,46.23,0.069906
64,SMCI,34.10,2026-02-10,2026-03-10,32.39,38.30,38.32,0.065591
9,CELH,48.05,2026-02-10,2026-03-10,43.78,49.48,41.53,-0.064932
70,TMDX,136.95,2026-02-10,2026-03-10,118.21,136.73,129.59,-0.064062
23,ENPH,50.05,2026-02-10,2026-03-10,46.14,49.11,45.34,-0.063670


In [ ]:
run_model_nn('stock', '0123').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
204,PUBM,12.82,2025-07-03,2025-07-31,12.64,16.44,14.37,0.129745
375,PI,115.86,2025-07-03,2025-07-31,116.02,144.76,128.19,0.119080
17,INTA,48.11,2025-07-03,2025-07-31,45.99,55.07,59.27,0.110857
352,ABR,10.90,2025-07-03,2025-07-31,10.53,12.98,12.60,0.104281
144,LFMD,12.50,2025-07-03,2025-07-31,11.93,14.68,14.55,0.097600
378,RCAT,7.08,2025-07-03,2025-07-31,5.73,7.56,5.93,-0.095104
10,ARQT,13.82,2025-07-03,2025-07-31,13.37,16.73,15.27,0.094308
348,CRM,272.15,2025-07-03,2025-07-31,267.03,319.47,305.57,0.092620
283,NTGR,29.41,2025-07-03,2025-07-31,28.97,35.49,31.90,0.092146
150,POWL,217.66,2025-07-03,2025-07-31,204.27,250.88,256.42,0.089727


In [ ]:
run_model_nn_cmd('buystocks', '0402', False, 500, 250).head(10)
# show_prediction('buystocks', '0508').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
342,OPEN,4.78,2026-02-04,2027-01-20,4.40,9.34,9.02,0.587169
46,EL,117.68,2026-02-04,2027-01-20,116.14,209.73,219.13,0.543734
156,INTC,47.65,2026-02-04,2027-01-20,45.20,88.05,86.21,0.535222
321,AEVA,13.06,2026-02-04,2027-01-20,11.22,23.51,24.11,0.501787
115,VOD,15.61,2026-02-04,2027-01-20,15.69,27.03,26.85,0.485586
93,CHGG,0.70,2026-02-04,2027-01-20,0.59,1.29,1.23,0.480952
228,SLDP,3.99,2026-02-04,2027-01-20,3.07,7.61,6.99,0.476190
219,PAGS,10.78,2026-02-04,2027-01-20,8.18,20.82,18.32,0.463203
324,ENPH,50.22,2026-02-04,2027-01-20,48.44,96.73,74.20,0.456060
348,REMX,84.97,2026-02-04,2027-01-20,82.66,162.47,123.48,0.446040


In [ ]:
run_model_nn_cmd('bluechipstock', '0117', False, 500, 250).head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
5,AVGO,328.46,2026-01-29,2027-01-14,287.62,608.10,568.04,0.485478
32,ORCL,164.71,2026-01-29,2027-01-14,152.25,315.91,249.25,0.451865
21,INTC,48.06,2026-01-29,2027-01-14,43.10,83.98,73.79,0.393189
17,DIS,111.24,2026-01-29,2027-01-14,106.58,174.07,155.86,0.308013
6,BA,234.45,2026-01-29,2027-01-14,223.97,336.86,318.81,0.250643
22,IBM,304.70,2026-01-29,2027-01-14,292.43,439.38,406.63,0.245422
39,UNH,292.80,2026-01-29,2027-01-14,151.07,325.39,187.66,-0.243944
18,GOOGL,335.86,2026-01-29,2027-01-14,301.50,478.89,438.51,0.209730
13,CSCO,78.73,2026-01-29,2027-01-14,74.46,108.06,98.34,0.189127
30,NVDA,191.13,2026-01-29,2027-01-14,150.83,254.82,273.16,0.183854


In [ ]:
run_model_nn_cmd('pennystock', '0122', False, 500, 250).head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
38,ONDS,12.55,2026-01-21,2027-01-06,9.32,24.94,23.65,0.538114
83,RUN,17.82,2026-01-21,2027-01-06,16.38,35.13,30.49,0.533857
101,IHS,7.68,2026-01-21,2027-01-06,6.50,15.12,13.60,0.528646
44,PGY,21.56,2026-01-21,2027-01-06,18.30,42.49,37.51,0.519790
86,SMR,19.75,2026-01-21,2027-01-06,12.70,38.35,37.51,0.494684
9,BGSF,5.62,2026-01-21,2027-01-06,5.19,10.81,8.67,0.463227
98,GPUS,0.28,2026-01-21,2027-01-06,0.05,0.36,0.05,-0.452381
36,NEPH,4.56,2026-01-21,2027-01-06,3.85,8.87,7.13,0.451023
62,VSAT,42.64,2026-01-21,2027-01-06,35.88,81.16,68.12,0.447467
74,EOSE,17.53,2026-01-21,2027-01-06,11.49,31.87,32.30,0.438677


In [ ]:
import sys
sys.path.append('/content')
!unzip drive/MyDrive/colab/stock_prediction_r.zip
import stock_prediction.stock_prediction.stock_predictor_lib
from stock_prediction.stock_prediction.stock_predictor_lib import run_model_nn, show_prediction, StockPredictorLib, Config
colab_path = 'content/drive/MyDrive/colab'

In [11]:
techcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
colab_path = 'content/drive/MyDrive/colab'
techcfg.model_path = f'/{colab_path}/models/transformer/techstock_1220'
techdf = run_model_nn(f'/{colab_path}/tickers/techstocktickers.csv', cfg=techcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/techstock_1220_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/techstock_1220_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/techstock_1220_close_model.pth'}


In [12]:
techdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
48,MU,375.85,2026-02-10,2026-03-10,227.85,382.23,223.76,-0.260485
44,MNDY,76.30,2026-02-10,2026-03-10,72.68,108.91,99.80,0.229314
11,COHR,230.74,2026-02-10,2026-03-10,172.44,237.92,138.73,-0.206770
63,RMBS,107.14,2026-02-10,2026-03-10,70.81,110.35,80.81,-0.184960
68,TEAM,96.41,2026-02-10,2026-03-10,92.85,126.74,122.82,0.183868
61,PUBM,6.84,2026-02-10,2026-03-10,6.75,9.32,8.00,0.173002
34,HUBS,239.07,2026-02-10,2026-03-10,215.79,318.56,304.50,0.169602
77,UI,717.46,2026-02-10,2026-03-10,486.38,727.43,588.56,-0.162615
49,NET,180.16,2026-02-10,2026-03-10,135.93,185.46,133.18,-0.158951
76,U,29.05,2026-02-10,2026-03-10,21.34,30.65,21.81,-0.153184


In [13]:
techrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
techrcfg.model_path = f'/{colab_path}/models/rolling/techstock_0109'
techrdf = run_model_nn(f'/{colab_path}/tickers/techstocktickers.csv', cfg=techrcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/techstock_0109_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/techstock_0109_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/techstock_0109_close_model.pth'}


In [14]:
techrdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
59,PI,113.58,2026-02-10,2026-03-10,109.78,143.28,143.36,0.163409
1,ACLS,88.05,2026-02-10,2026-03-10,81.30,102.35,114.80,0.129850
51,NVCR,10.56,2026-02-10,2026-03-10,10.11,14.19,10.95,0.112689
40,KD,11.69,2026-02-10,2026-03-10,10.69,14.29,13.94,0.109780
5,APP,475.24,2026-02-10,2026-03-10,363.88,483.16,437.99,-0.098680
30,GTLB,31.94,2026-02-10,2026-03-10,30.96,37.98,36.25,0.097788
32,HIMS,18.31,2026-02-10,2026-03-10,17.80,22.42,19.73,0.091389
75,TWLO,120.75,2026-02-10,2026-03-10,100.79,131.62,99.94,-0.082540
77,UI,717.46,2026-02-10,2026-03-10,598.61,741.56,639.57,-0.080209
16,DDOG,130.56,2026-02-10,2026-03-10,96.04,136.59,129.07,-0.076542


In [ ]:
defetfcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
defetfcfg.model_path = f'/{colab_path}/models/rolling/etf_1221'
etfdf = run_model_nn(f'/{colab_path}/tickers/etftickers.csv', cfg=defetfcfg)
etfdf.head(10)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/etf_1221_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/etf_1221_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/etf_1221_close_model.pth'}


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
22,XLC,116.91,2026-01-27,2026-02-24,114.02,121.17,127.73,0.034756
21,XLB,49.99,2026-01-27,2026-02-24,48.59,53.51,52.07,0.028006
0,ARKF,46.05,2026-01-27,2026-02-24,45.11,49.12,47.49,0.025841
31,XME,128.38,2026-01-27,2026-02-24,125.85,138.44,128.85,0.020772
19,URA,57.85,2026-01-27,2026-02-24,56.05,63.46,57.59,0.020455
3,DIA,490.07,2026-01-27,2026-02-24,479.53,513.22,507.28,0.020283
28,XLU,43.41,2026-01-27,2026-02-24,40.92,44.09,42.89,-0.017891
16,QQQ,631.13,2026-01-27,2026-02-24,615.90,651.98,656.09,0.016151
8,IXC,46.34,2026-01-27,2026-02-24,45.23,48.58,47.37,0.015537
15,JXI,81.94,2026-01-27,2026-02-24,77.62,83.46,81.52,-0.013099


In [ ]:
tvolcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
tvolcfg.model_path = f'/{colab_path}/models/transformer/pennystock_1226'
tvoldf = run_model_nn(f'/{colab_path}/tickers/pennystocktickers.csv', cfg=tvolcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/pennystock_1226_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/pennystock_1226_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/pennystock_1226_close_model.pth'}


In [ ]:
tvoldf[tvoldf['predicted_return']> 0].head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
27,INLX,8.11,2026-01-28,2026-02-25,6.26,68.92,9.52,2.481299
2,AMSSY,5.16,2026-01-27,2026-02-24,5.11,7.92,14.22,0.760336
110,IBRX,5.59,2026-01-28,2026-02-25,5.07,10.45,8.74,0.446631
99,SQNS,5.16,2026-01-28,2026-02-25,4.24,8.32,6.92,0.258398
13,CLOV,2.32,2026-01-28,2026-02-25,2.17,3.82,2.66,0.242816
109,AUID,1.48,2026-01-28,2026-02-25,1.34,1.85,2.20,0.213964
56,SST,4.17,2026-01-28,2026-02-25,3.89,5.39,5.35,0.169464
20,GOGO,4.53,2026-01-28,2026-02-25,4.18,5.53,6.04,0.158940
74,EOSE,15.68,2026-01-28,2026-02-25,13.51,21.61,18.69,0.143920
10,BLNK,0.80,2026-01-28,2026-02-25,0.67,1.02,1.03,0.133333


In [ ]:
tvolrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
tvolrcfg.model_path = f'/{colab_path}/models/rolling/pennystock_0114'
tvolrdf = run_model_nn(f'/{colab_path}/tickers/pennystocktickers.csv', cfg=tvolrcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/pennystock_0114_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/pennystock_0114_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/pennystock_0114_close_model.pth'}


In [ ]:
tvolrdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
82,WKEY,8.72,2026-01-14,2026-02-11,6.54,13.68,10.52,0.175076
39,INVZ,1.06,2026-01-14,2026-02-11,0.93,1.55,1.22,0.163522
32,GPUS,0.32,2026-01-14,2026-02-11,0.24,0.34,0.25,-0.135417
48,NEPH,4.40,2026-01-14,2026-02-11,4.14,5.58,5.25,0.134091
15,CARS,11.97,2026-01-14,2026-02-11,11.23,14.27,14.62,0.117238
9,BBAI,6.12,2026-01-14,2026-02-11,4.24,6.48,5.78,-0.101307
79,UDMY,5.14,2026-01-14,2026-02-11,4.83,6.15,5.99,0.100519
81,VSAT,42.03,2026-01-14,2026-02-11,33.30,44.14,36.34,-0.097629
30,GRAB,4.65,2026-01-14,2026-02-11,4.53,5.25,5.46,0.092473
68,SFIX,5.24,2026-01-14,2026-02-11,4.01,5.67,4.65,-0.088422


In [ ]:
bctcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
bctcfg.model_path = f'/{colab_path}/models/transformer/bluechipstock_1010'
bctdf = run_model_nn(f'/{colab_path}/tickers/bluechipstocktickers.csv', cfg=bctcfg)
bctdf.head(10)

In [ ]:
bcrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
bcrcfg.model_path = f'/{colab_path}/models/rolling/bluechipstock_0117'
bcrdf = run_model_nn(f'/{colab_path}/tickers/bluechipstocktickers.csv', cfg=bcrcfg)
bcrdf.head(10)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/bluechipstock_0117_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/bluechipstock_0117_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/bluechipstock_0117_close_model.pth'}


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
29,NKE,64.38,2026-01-16,2026-02-13,62.87,74.50,69.46,0.070881
16,DHR,235.99,2026-01-16,2026-02-13,230.89,256.36,258.42,0.053251
25,LLY,1038.40,2026-01-16,2026-02-13,999.85,1136.99,1135.79,0.050536
17,DIS,111.20,2026-01-16,2026-02-13,108.60,120.33,118.70,0.042056
11,CRM,227.11,2026-01-16,2026-02-13,220.41,251.22,236.99,0.040054
12,COST,963.61,2026-01-16,2026-02-13,923.90,1031.77,1050.03,0.039736
31,NOW,127.31,2026-01-16,2026-02-13,121.23,142.72,133.10,0.039588
0,AAPL,255.53,2026-01-16,2026-02-13,253.88,283.29,259.19,0.038834
2,ADBE,296.12,2026-01-16,2026-02-13,284.27,337.85,298.30,0.036089
7,BKNG,5115.91,2026-01-16,2026-02-13,4977.34,5434.66,5410.26,0.030919


In [ ]:
buystkcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
buystkcfg.model_path = f'/{colab_path}/models/transformer/buystocks_0123'
buystkcfg.model.window_size = 500
buystkcfg.model.future_days = 250
buystkdf = run_model_nn(f'/{colab_path}/tickers/buystockstickers.csv', cfg=buystkcfg)

In [ ]:
buystkdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
28,BNED,9.43,2026-01-22,2027-01-07,8.50,10.09,248.54,8.442559
94,CHGG,0.79,2026-01-22,2027-01-07,0.78,11.47,5.72,6.582278
269,CRNC,11.49,2026-01-22,2027-01-07,11.16,124.59,99.59,5.827386
10,SFIX,5.22,2026-01-22,2027-01-07,5.04,50.66,44.06,5.370370
86,RIVN,16.32,2026-01-22,2027-01-07,13.62,118.22,99.29,3.720792
18,INVZ,1.13,2026-01-22,2027-01-07,1.10,7.17,7.46,3.640118
55,LCID,11.47,2026-01-22,2027-01-07,11.07,74.41,57.85,3.165359
35,UP,0.83,2026-01-22,2027-01-07,0.61,6.72,1.30,2.465863
17,OUST,27.24,2026-01-22,2027-01-07,11.33,132.54,112.54,2.137665
144,PTON,5.81,2026-01-22,2027-01-07,5.82,26.19,18.97,1.924842


In [ ]:
buystkrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
buystkrcfg.model_path = f'/{colab_path}/models/rolling/buystocks_1213'
buystkrcfg.model.window_size = 500
buystkrcfg.model.future_days = 250
buystkrdf = run_model_nn(f'/{colab_path}/tickers/buystockstickers.csv', cfg=buystkrcfg)

In [ ]:
buystkrdf.head(10)

In [ ]:
stkcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
stkcfg.model_path = f'/{colab_path}/models/transformer/stock_0105'
stkdf = run_model_nn(f'/{colab_path}/tickers/stocktickers.csv', cfg=stkcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/stock_0105_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/stock_0105_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/stock_0105_close_model.pth'}


In [ ]:
stkdf[stkdf['predicted_return'] > 0].head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
371,SQNS,3.62,2026-02-09,2026-03-09,3.26,5.10,5.10,0.239411
29,BNED,8.70,2026-02-09,2026-03-09,8.07,12.64,10.85,0.209195
251,ENVX,6.57,2026-02-09,2026-03-09,5.84,7.33,10.46,0.198884
36,UP,0.69,2026-02-09,2026-03-09,0.58,1.12,0.78,0.198068
187,PUBM,6.55,2026-02-09,2026-03-09,6.25,9.23,7.95,0.192366
97,CHGG,0.75,2026-02-09,2026-03-09,0.68,1.06,0.84,0.146667
65,GLOB,58.39,2026-02-09,2026-03-09,52.20,72.62,74.67,0.138837
108,TWLO,115.44,2026-02-09,2026-03-09,108.31,126.86,153.15,0.121275
357,ETHA,15.80,2026-02-09,2026-03-09,14.78,17.91,19.88,0.109072
30,AI,11.60,2026-02-09,2026-03-09,10.99,15.29,12.13,0.103736


In [ ]:
stkrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
stkrcfg.model_path = f'/{colab_path}/models/rolling/stock_0102'
stkrdf = run_model_nn(f'/{colab_path}/tickers/stocktickers.csv', cfg=stkrcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/stock_0102_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/stock_0102_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/stock_0102_close_model.pth'}


In [ ]:
stkrdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
97,CHGG,0.70,2026-02-04,2026-03-04,0.67,1.04,0.83,0.209524
49,IREN,46.50,2026-02-04,2026-03-04,41.95,62.18,63.56,0.202079
346,KVYO,20.23,2026-02-04,2026-03-04,19.11,24.98,27.38,0.177624
339,QBTS,19.20,2026-02-04,2026-03-04,16.83,25.51,22.33,0.122743
211,PEGA,39.21,2026-02-04,2026-03-04,36.20,52.19,42.58,0.113406
22,STX,420.87,2026-02-04,2026-03-04,389.84,470.70,545.12,0.113297
335,RGTI,16.40,2026-02-04,2026-03-04,15.37,21.74,17.33,0.106504
263,LUNR,16.40,2026-02-04,2026-03-04,13.68,20.90,19.86,0.106504
353,NUTX,139.62,2026-02-04,2026-03-04,129.14,175.25,158.83,0.105907
381,BITF,2.02,2026-02-04,2026-03-04,1.90,2.71,2.07,0.102310


In [ ]:
glbcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
glbcfg.model_path = f'/{colab_path}/models/transformer/global_0105'
glbdf = run_model_nn(f'/{colab_path}/tickers/globaltickers.csv', cfg=glbcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/global_0105_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/global_0105_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/global_0105_close_model.pth'}


In [ ]:
glbdf[glbdf['predicted_return']>0].head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
59,TGLS,50.08,2026-02-05,2026-03-05,48.79,58.36,54.47,0.075745
39,LOMA,10.75,2026-02-05,2026-03-05,10.48,12.81,11.23,0.070388
49,POAHF,42.11,2026-02-04,2026-03-04,39.22,48.65,46.60,0.064434
1,ADDDF,175.41,2026-02-04,2026-03-04,173.03,185.30,198.35,0.057864
46,ONON,43.06,2026-02-05,2026-03-05,42.30,48.44,45.02,0.050937
25,FQVLF,26.20,2026-02-05,2026-03-05,25.92,28.72,27.76,0.048346
50,RNMBF,1845.00,2026-02-05,2026-03-05,1820.37,2048.90,1931.47,0.048011
51,RYCEF,16.62,2026-02-05,2026-03-05,15.98,18.75,17.52,0.047934
40,LVMUY,126.55,2026-02-05,2026-03-05,121.15,131.81,142.44,0.041486
38,LYB,53.11,2026-02-05,2026-03-05,49.78,59.44,55.12,0.031444


In [ ]:
glbrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
glbrcfg.model_path = f'/{colab_path}/models/rolling/global_1210'
glbrdf = run_model_nn(f'/{colab_path}/tickers/globaltickers.csv', cfg=glbrcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/global_1210_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/global_1210_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/global_1210_close_model.pth'}


In [ ]:
glbrdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
34,KNNGF,66.45,2025-10-23,2025-11-20,54.51,66.45,58.78,-0.098370
57,SE,124.22,2025-12-10,2026-01-07,120.04,137.44,150.25,0.094107
37,RACE,364.04,2025-12-10,2026-01-07,364.08,407.93,406.34,0.078957
51,PROSF,60.00,2025-12-08,2026-01-05,56.53,69.72,66.23,0.069333
24,EVKIY,7.54,2025-12-09,2026-01-06,7.39,8.82,7.57,0.051282
35,MELI,1970.73,2025-12-10,2026-01-07,1956.95,2137.48,2095.23,0.046932
23,ESEA,59.00,2025-12-10,2026-01-07,56.62,63.81,64.52,0.044915
56,SAP,245.88,2025-12-10,2026-01-07,236.16,263.98,269.01,0.042717
61,TGLS,53.41,2025-12-10,2026-01-07,45.93,59.29,48.65,-0.039693
18,ENGGY,7.96,2025-12-10,2026-01-07,7.70,8.42,8.61,0.035595


In [ ]:
ctcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
ctcfg.model_path = f'/{colab_path}/models/transformer/country_1211'
ctdf = run_model_nn(f'/{colab_path}/tickers/countrytickers.csv', cfg=ctcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/country_1211_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/country_1211_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/country_1211_close_model.pth'}


In [ ]:
ctdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
4,EPHE,26.73,2026-02-06,2026-03-06,25.15,26.83,24.98,-0.040279
24,XCEM,42.88,2026-02-06,2026-03-06,40.69,42.86,41.31,-0.029384
8,EWJ,89.40,2026-02-06,2026-03-06,84.34,89.46,87.43,-0.025988
19,KEMQ,25.92,2026-02-06,2026-03-06,25.85,27.15,26.75,0.025592
18,INDA,53.28,2026-02-06,2026-03-06,52.89,55.35,55.05,0.021584
21,THD,65.75,2026-02-06,2026-03-06,61.27,66.26,65.70,-0.020380
16,IDX,15.55,2026-02-06,2026-03-06,15.41,16.19,15.97,0.019721
11,EWP,56.46,2026-02-06,2026-03-06,53.17,57.02,55.97,-0.019011
0,ARGT,93.16,2026-02-06,2026-03-06,91.49,98.77,94.31,0.018212
20,KWT,37.25,2026-02-06,2026-03-06,37.13,37.71,35.01,-0.017002


In [ ]:
valtcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
valtcfg.model_path = f'/{colab_path}/models/transformer/valuestock_1018'
valtdf = run_model_nn(f'valuestocktickers.csv', cfg=valtcfg)


In [ ]:
valtdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
71,POWL,355.91,2025-11-12,2025-12-10,318.63,406.16,497.69,0.144934
16,CC,12.51,2025-11-12,2025-12-10,11.58,15.90,14.90,0.129230
7,ARE,53.86,2025-11-12,2025-12-10,52.99,59.89,66.73,0.111586
91,UAL,99.94,2025-11-12,2025-12-10,91.43,119.48,121.23,0.107798
14,BXC,57.66,2025-11-12,2025-12-10,55.51,69.67,63.39,0.090126
44,GLW,88.71,2025-11-12,2025-12-10,86.23,108.45,94.80,0.087739
74,RH,160.72,2025-11-12,2025-12-10,156.87,193.12,172.52,0.083686
37,F,13.45,2025-11-12,2025-12-10,12.72,13.75,10.53,-0.083024
58,MOD,147.99,2025-11-12,2025-12-10,142.12,179.25,154.26,0.071311
39,FICO,1778.19,2025-11-12,2025-12-10,1648.44,2256.72,1784.05,0.066480


In [ ]:
valrcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
valrcfg.model_path = f'/{colab_path}/models/rolling/valuestock_0106'
valrdf = run_model_nn(f'/{colab_path}/tickers/valuestocktickers.csv', cfg=valrcfg)


Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/valuestock_0106_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/valuestock_0106_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/valuestock_0106_close_model.pth'}


In [ ]:
valrdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
4,ANF,97.66,2026-02-06,2026-03-06,88.33,114.06,114.75,0.082463
89,TWI,11.06,2026-02-06,2026-03-06,10.74,12.06,13.04,0.080169
63,OLN,24.83,2026-02-06,2026-03-06,21.66,25.23,22.68,-0.066049
82,STRL,401.29,2026-02-06,2026-03-06,342.84,411.94,371.96,-0.064068
83,SWK,89.39,2026-02-06,2026-03-06,81.00,91.61,78.89,-0.062162
67,PETS,2.88,2026-02-06,2026-03-06,2.76,3.21,3.13,0.053241
71,POWL,584.22,2026-02-06,2026-03-06,564.52,657.37,621.67,0.051864
16,CC,18.32,2026-02-06,2026-03-06,16.19,19.71,16.28,-0.050582
45,GWW,1197.65,2026-02-06,2026-03-06,1017.33,1236.10,1168.14,-0.047699
28,CSL,403.86,2026-02-06,2026-03-06,361.30,419.62,373.08,-0.047525


In [ ]:
commtcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
commtcfg.model_path = f'/{colab_path}/models/transformer/commodity_0109'
commtdf = run_model_nn(f'/{colab_path}/tickers/commoditytickers.csv', cfg=commtcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/commodity_0109_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/commodity_0109_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/commodity_0109_close_model.pth'}


In [ ]:
commtdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
8,SLV,75.86,2026-02-09,2026-03-09,58.45,75.10,60.05,-0.149310
5,GLD,466.41,2026-02-09,2026-03-09,456.95,470.61,405.17,-0.047526
7,PALL,158.26,2026-02-09,2026-03-09,139.13,168.84,154.04,-0.026897
11,TLT,87.51,2026-02-09,2026-03-09,86.67,89.79,89.58,0.013370
0,DBC,24.24,2026-02-09,2026-03-09,23.72,24.99,24.73,0.009901
6,ISHG,76.94,2026-02-09,2026-03-09,75.66,77.32,76.19,-0.007148
13,UUP,26.78,2026-02-09,2026-03-09,26.68,27.34,26.88,0.006970
9,SOYB,22.94,2026-02-09,2026-03-09,22.31,24.03,22.94,0.006684
15,WEAT,20.65,2026-02-09,2026-03-09,20.01,21.24,21.07,0.005973
4,FXY,58.97,2026-02-09,2026-03-09,58.13,60.66,57.44,-0.003844


In [ ]:
cryptcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
cryptcfg.model_path = f'/{colab_path}/models/transformer/crypto_1021'
cryptdf = run_model_nn(f'/{colab_path}/tickers/cryptotickers.csv', cfg=cryptcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/crypto_1021_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/crypto_1021_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/crypto_1021_close_model.pth'}


In [ ]:
cryptdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
15,XLM-USD,0.23,2025-11-21,2025-12-19,0.22,0.28,0.29,0.144928
6,DOGE-USD,0.14,2025-11-21,2025-12-19,0.14,0.16,0.18,0.142857
3,ADA-USD,0.41,2025-11-21,2025-12-19,0.39,0.48,0.53,0.138211
4,XRP-USD,1.96,2025-11-21,2025-12-19,1.93,2.34,2.39,0.132653
5,DOT-USD,2.37,2025-11-21,2025-12-19,2.25,3.10,2.64,0.123769
11,ICP-USD,4.36,2025-11-21,2025-12-19,3.64,4.84,5.74,0.087156
12,AVAX-USD,13.32,2025-11-21,2025-12-19,8.87,16.84,11.20,-0.076326
17,MATIC-USD,0.22,2025-03-24,2025-04-21,0.15,0.26,0.20,-0.075758
18,ATOM-USD,2.53,2025-11-21,2025-12-19,2.31,3.29,2.50,0.067194
16,AAVE-USD,157.80,2025-11-21,2025-12-19,152.88,185.84,162.92,0.059654


In [ ]:
cryprcfg = Config.from_file('./stock_prediction/configs/rolling.yaml')
cryprcfg.model_path = f'/{colab_path}/models/rolling/crypto_0109'
cryprdf = run_model_nn(f'/{colab_path}/tickers/cryptotickers.csv', cfg=cryprcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/rolling/crypto_0109_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/rolling/crypto_0109_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/rolling/crypto_0109_close_model.pth'}


In [ ]:
cryprdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
1,ETH-USD,2126.48,2026-02-09,2026-03-09,1876.01,2825.73,2225.94,0.085939
0,BTC-USD,70747.74,2026-02-09,2026-03-09,63345.03,93437.51,72368.39,0.079662
16,AAVE-USD,113.54,2026-02-09,2026-03-09,104.39,145.90,114.33,0.070460
3,ADA-USD,0.27,2026-02-09,2026-03-09,0.25,0.34,0.27,0.061728
17,MATIC-USD,0.22,2025-03-24,2025-04-21,0.19,0.24,0.19,-0.060606
2,SOL-USD,87.70,2026-02-09,2026-03-09,81.34,109.60,87.74,0.059217
7,FIL-USD,0.94,2026-02-09,2026-03-09,0.85,1.12,0.93,0.028369
13,LINK-USD,8.91,2026-02-09,2026-03-09,8.10,10.62,8.63,0.023195
11,ICP-USD,2.45,2026-02-09,2026-03-09,2.22,2.65,2.31,-0.023129
10,BCH-USD,534.34,2026-02-09,2026-03-09,405.40,678.99,483.72,-0.021778


In [ ]:
etftcfg = Config.from_file('./stock_prediction/configs/transformer.yaml')
etftcfg.model_path = f'/{colab_path}/models/transformer/etf_0112'
etftdf = run_model_nn(f'/{colab_path}/tickers/etftickers.csv', cfg=etftcfg)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/etf_0112_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/etf_0112_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/etf_0112_close_model.pth'}


In [ ]:
etftdf.head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
19,URA,51.76,2026-01-12,2026-02-09,41.08,52.38,41.30,-0.132148
4,DXYZ,30.37,2026-01-12,2026-02-09,29.50,34.27,38.67,0.124355
31,XME,120.71,2026-01-12,2026-02-09,108.28,122.34,93.18,-0.105846
7,IPO,48.51,2026-01-12,2026-02-09,44.65,48.64,43.78,-0.058132
12,IWO,344.97,2026-01-12,2026-02-09,310.29,351.61,317.84,-0.053309
23,XLE,46.38,2026-01-12,2026-02-09,46.01,49.96,48.83,0.040678
13,IYZ,33.79,2026-01-12,2026-02-09,30.00,35.06,32.47,-0.037881
14,JETS,29.26,2026-01-12,2026-02-09,26.47,29.53,29.07,-0.030873
30,XLY,124.51,2026-01-12,2026-02-09,117.95,128.40,116.84,-0.027682
18,SPY,695.61,2026-01-12,2026-02-09,690.33,729.39,712.75,0.021870


In [ ]:
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd
from google.auth import default


auth.authenticate_user()


creds, _ = default()
gc = gspread.authorize(creds)
# gc = gspread.authorize(GoogleCredentials.get_application_default())


In [ ]:
# Open your spreadsheet by name
spreadsheet = gc.open('Allocation')
# Or by URL: spreadsheet = gc.open_by_url('Your Spreadsheet URL')

# Select the worksheet (e.g., the first sheet)
worksheet = spreadsheet.worksheet("Buy")

# Get all values as a list of lists
data = worksheet.get_all_values()

# Convert to a Pandas DataFrame
sheetdf = pd.DataFrame(data[1:], columns=data[0]) # Assuming first row is header


In [ ]:
df = sheetdf[['Ticker']].copy()

# Apply the extraction function first, then filter based on the result
def extract_ticker(ticker_str):
    # Convert to string before splitting
    ticker_str = str(ticker_str)
    parts = ticker_str.split(':')
    if len(parts) > 1:
        return parts[1].strip() # Use strip to remove leading/trailing whitespace
    return ticker_str.strip() # Return the original string if no colon, also strip whitespace

df['Ticker'] = df['Ticker'].apply(extract_ticker)
# Remove BTCUSD and add BTC-USD, ETH-USD, ADA-USD, SOL-USD in Ticker
df = df[df['Ticker'] != 'BTCUSD']
df = pd.concat([df, pd.DataFrame({'Ticker': ['BTC-USD', 'ETH-USD', 'ADA-USD', 'SOL-USD']})], ignore_index=True)
df.to_csv(f'/{colab_path}/tickers/stocktickers.csv', index=False)
df.to_csv(f'/{colab_path}/tickers/buystockstickers.csv', index=False)

In [ ]:
# Only have Tickers column

run_model_nn('buystocks', '0402', False, 500, 250, copy_tickers=False).head(10)


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
333,QBTS,34.25,2025-10-08,2026-09-23,32.44,71.01,65.57,0.644964
96,CHGG,1.42,2025-10-08,2026-09-23,1.40,2.82,2.74,0.633803
97,SLP,16.47,2025-10-08,2026-09-23,16.09,32.07,31.69,0.616070
18,OUST,30.65,2025-10-08,2026-09-23,29.45,61.59,56.80,0.607830
9,HIMS,58.17,2025-10-08,2026-09-23,49.10,110.63,110.40,0.547934
252,LYFT,21.16,2025-10-08,2026-09-23,19.66,41.40,36.93,0.543636
6,DOMO,14.74,2025-10-08,2026-09-23,12.98,29.59,25.17,0.531886
326,ROOT,81.73,2025-10-08,2026-09-23,76.40,156.09,141.76,0.526367
122,LZ,9.97,2025-10-08,2026-09-23,9.63,19.32,16.65,0.524574
276,CRNC,11.96,2025-10-08,2026-09-23,11.62,21.43,21.43,0.518395


In [ ]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +180 -delete -print

sending incremental file list
buystocks_0709_predictions.csv

sent 18,897 bytes  received 35 bytes  37,864.00 bytes/sec
total size is 18,770  speedup is 0.99


In [ ]:
# Copy zip file from /content/drive/MyDrive/colab/ to /tmp/stock_cache.zip
# !cp /content/drive/MyDrive/colab/stock_cache.zip /tmp/stock_cache.zip
# update zip /tmp/stock_cache.zip from files in folder /tmp/stock_cache
!zip -ur /tmp/stock_cache.zip /tmp/stock_cache
!cp /tmp/stock_cache.zip /content/drive/MyDrive/colab/